In [1]:
import pandas as pd
import numpy as np
import numpy
import matplotlib.pyplot as plt
import pandas
import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
from pandas import DataFrame
from pandas import concat
from math import sqrt
from matplotlib import pyplot

In [24]:
confirmed = pd.read_csv('../Zaida/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('../Zaida/time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('../Zaida/time_series_covid19_recovered_global.csv')

In [25]:
confirmed = confirmed.melt(id_vars=['Country/Region','Province/State','Lat','Long'],var_name="Date",value_name="Confirmed")
deaths = deaths.melt(id_vars=['Country/Region','Province/State','Lat','Long'],var_name="Date",value_name="Deaths")
recovered = recovered.melt(id_vars=['Country/Region','Province/State','Lat','Long'],var_name="Date",value_name="Recovered")
confirmed['Date'] = pd.to_datetime(confirmed['Date'])
deaths['Date'] = pd.to_datetime(deaths['Date'])
recovered['Date'] = pd.to_datetime(recovered['Date'])

df = pd.merge(confirmed,deaths,on=['Country/Region','Province/State','Lat','Long','Date'])
df = pd.merge(df,recovered,on=['Country/Region','Province/State','Lat','Long','Date']).sort_values(by=['Country/Region','Date'])
df = df[df['Province/State'].isnull()] #only take european countries from provinces

In [26]:
df

,Country/Region,Province/State,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Afghanistan,NaN,33.0,65.0,2020-01-22,0,0,0
249,Afghanistan,NaN,33.0,65.0,2020-01-23,0,0,0
498,Afghanistan,NaN,33.0,65.0,2020-01-24,0,0,0
747,Afghanistan,NaN,33.0,65.0,2020-01-25,0,0,0
996,Afghanistan,NaN,33.0,65.0,2020-01-26,0,0,0
1245,Afghanistan,NaN,33.0,65.0,2020-01-27,0,0,0
1494,Afghanistan,NaN,33.0,65.0,2020-01-28,0,0,0
1743,Afghanistan,NaN,33.0,65.0,2020-01-29,0,0,0
1992,Afghanistan,NaN,33.0,65.0,2020-01-30,0,0,0
2241,Afghanistan,NaN,33.0,65.0,2020-01-31,0,0,0
